## MSc Data Analytics - Capstone Project

#### Predictive Analysis in the Coffee Market: Using Deep Learning to predict coffee prices.
Student id: 2020274 Clarissa Cardoso


## Introduction
This notebook aims to analyse historical coffee price indices and develop a predictive model for future price trends in the green coffee market. The focus is on using data from the ICO (International Coffee Organization), particularly the composite prices indec (I-CIP), that combines prices of Colombian Milds, Other Milds, Brazilian Naturals, and Robustas.

### Dataset:
The dataset used in this analysis consists of historical coffee price data, with daily observations for business days. Prices are expressed in cents of USD per lb. and stated on a
differential basis to the New York and London futures exchanges (https://icocoffee.org/wp-content/uploads/2023/01/icc-105-17-r1e-rules-indicator-prices.pdf)

The data utilized in this project is sourced from the International Coffee Organization's (ICO) Public Market Information (https://ico.org/resources/public-market-information/), which provides the I-CIP values free of charge.

For the early stages of the experimentation, 1 year worth of data was available to collect, from 01Feb23 to 29Feb24, which is present on a separate notebook (2020274_capstone_EDA_Models 2.ipynb). In this notebook, recent data from March to September 2024 were added to expand insights and feed more datapoints to modelling stage. 


### Objectives:
1. Clean and preprocess the dataset for missing values and inconsistencies.
2. Explore the time-series behavior of coffee prices through visualizations.
3. Implement various forecasting models to predict future price trends, including traditional statistical models (e.g., ARIMA/Sarima) and deep learning algorithms (e.g., LSTM neural networks).
4. Compare model performance using key metrics (e.g., RMSE, MAE).


### Expected Outcome:
By the end of this notebook, we will identify the best forecasting model for coffee prices and present actionable insights based on the findings.

        Forecasting: generate forecasts for future I-CIP values using the best-performing model(s) and visualize the results to facilitate interpretation and decision-making.
- 1 day
- 5 days = 1 week
- 21 days = 1 month


(- 63 days = 3 months (1 quarter))

### Importing relevant libraries for the project

In [ ]:
#importing libraries
import warnings
warnings.filterwarnings("ignore")

import pandas as pd #dataframes 
import numpy as np #linear algebra
import seaborn as sns #visualization
sns.set(color_codes=True)


import plotly.express as px
import plotly.graph_objects as go


import scipy.stats as stats #statistical resources

import matplotlib.pyplot as plt #visualisation 
%matplotlib inline 


from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import matplotlib as mpl

from sklearn.model_selection import train_test_split # importing function to split the data training and test.
from sklearn.preprocessing import MinMaxScaler # Import the MinMaxScaler module from sklearn.preprocessing library
from sklearn.linear_model import LinearRegression # importing to performe linear regression. 
from sklearn.metrics import make_scorer, r2_score # Importing from Metrics module
from sklearn.preprocessing import StandardScaler # standardize the data
from sklearn import metrics # Metrics module from scikit-learn
from sklearn.model_selection import GridSearchCV # importing for hyperparameter tunning
from sklearn.metrics import mean_squared_error # importing mse
from scipy.stats import shapiro

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential #last update in python causing dead kernel wehn importing keras functions?
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from scipy.interpolate import interp1d

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf



In [ ]:
import keras
import tensorflow as tf

print("Keras version:", keras.__version__)
print("TensorFlow version:", tf.__version__)


## cheking if keras/tensorflow are correclty installed 

# 1. Load data

For the early stages of this experimentation present on the first notebook (Models2_copy), 1 year worth of data was available to collect, from 01Feb23 to 29Feb24.

This section will review the original dataset compiled with data from feb 23 to feb 24.


A few thingsobserved when importing the raw files from the data source: 

- Column mismatch: Assuming all files have the same column names and order. This could lead to errors when merging DataFrames with different structures. 

The data for each month is published separetely. Originally the 4 first months had different colum labels for the same data 'ICO Composite' , while the following months was simpler version as 'I-CIP'. For that diverson it was not possible to simply merge all dataframes into one. Dta cleaning/manipulation techniques of renamimbg and reorganising them chronologically were adopeted to reach the final dataset for the first year of data alocated in the 'icip_data' below. 





In [ ]:
# Read the CSV file 
icip_data = pd.read_csv("icip_df.csv")

# View the first 5 rows
icip_data.head()

In [ ]:
icip_data.info()

Since then, ICO has released additional months that will be included in the main dataframe, considering the timeframe from march to september 2024 as a way to feed more data to the models with the expectation it could improve the results. These seven new files will be sorted by chronological order and have the same labels as the main one above. 


### Importing  additional data from March/24 to September/24 

In [ ]:
import os
# List all the files in the folder
os.listdir("icip_24") 

In [ ]:
#create for loop to import csv files from the folder with less comands.

# create an empty list to store dfs
dataframes = []

# path to folder where csv files are (in this case same directory)
folder_path = "icip_24"


# to import CSV starting from the third row, skipping the first two
def import_csv(filepath):
    return pd.read_csv(filepath, skiprows=2)

# Iterate through files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # Only consider CSV files
        file_path = os.path.join(folder_path, file)  # Construct the full file path
        dataframes.append(import_csv(file_path))  # Read CSV and append to list

In [ ]:
#check the lenght of the directory, how many files exist in the new folder
len(dataframes)

Chcking the heading of the files to undertand how features are allocated in this first stage before combining the new 7 months to main dataframe

The same issue appears with the heading names. So this time around it was decided to ignore the first 2 rows to avoid the unnamed labels and only import the actual data 

>Unnamed: 0	Unnamed: 1	Colombian	Unnamed: 3	Brazilian	Unnamed: 5


>0	NaN	I-CIP	NaN	Other Milds	NaN	Robusta


In [ ]:
#check if order of files correspond with the directory list, testing if loop is working
dataframes[5].head()

In [ ]:
print(dataframes)
#list of all dataframes

To continue the project is necessary to make 2 adjustments in the second directory:
- change the date format from " 06-Jun" to '%Y-%m-%d' format and apply this to all files in the "Unnamed: 0" collum which corresponds to date. This will enable a more smooth combination of the 2 dfs once all dates mantain the correct format. 

In [ ]:
# Test: print the first DataFrame to check if the transformation worked
print(dataframes[5].head())

The code below will use the month mapping to rearranje data in the first colum from corresponding to dates.
- first it will change the format from '**01-Jul**' to **''%Y-%m-%d''** through all months in the list of dataframes.
- then will replace the dataframes with the correct format and set as 'datetime' type. 
- print the correct year for all dataframes and show the correct label for 'date'

In [ ]:
# Function to transform the 'Unnamed: 0' date column for each DataFrame in the list and reorder columns
def transform_date(dataframes, year):
    month_mapping = {
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
        'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
        'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    
    # Iterate over each DataFrame in the list
    for i in range(len(dataframes)):
        df = dataframes[i]
        
        # Print the columns to inspect if 'Unnamed: 0' exists or if the name is different
        print(f"Columns in DataFrame {i}: {df.columns}")
        
        # Check if 'Unnamed: 0' exists, otherwise handle the column name differently
        if 'Unnamed: 0' in df.columns:
            # Apply the transformation to the 'Unnamed: 0' column to create full date strings
            df['date'] = df['Unnamed: 0'].apply(
                lambda x: '-'.join([str(year), month_mapping[x.split('-')[1]], x.split('-')[0]])
            )
            
            # Convert the 'Date' column to datetime format
            df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
            
            # Drop the original 'Unnamed: 0' column
            df.drop(columns=['Unnamed: 0'], inplace=True)
            
            # Reorder columns to place 'Date' first
            columns = ['date'] + [col for col in df.columns if col != 'date']
            dataframes[i] = df[columns]  # Replace the DataFrame with the reordered one
        else:
            print(f"'Unnamed: 0' column not found in DataFrame {i}")
    
    return dataframes

# Apply the function to the list of DataFrames
dataframes = transform_date(dataframes, 2024)

# Test: print the first DataFrame to check if the column reordering worked
print(dataframes[0].head())

## Splitting the date column to match main dataset

This function is mainly to add the last 2 columns matching the original dataset which contains the year and m,onth of each date. This can also be used for some of exploratory plots in next sectiosn. 

In [ ]:
# Function to add year and month columns to each DataFrame in the list
def add_year_month_columns(dataframes):
    for i in range(len(dataframes)):
        df = dataframes[i]
        
        # Extract the year and month from the 'Date' column
        df['year'] = df['date'].dt.year
        df['month'] = df['date'].dt.month
        
        # Replace the DataFrame in the list with the new columns added
        dataframes[i] = df
        
    return dataframes

# Apply the function to the list of DataFrames
dataframes = add_year_month_columns(dataframes)

# checking if transformation worked in the dataframes list:
dataframes[0].head()

### Define chronologic order for dataframe

In [ ]:
# Define the list of DataFrames in the desired order
dfs_in_order = [dataframes[3],dataframes[2],dataframes[4],dataframes[6],dataframes[5],dataframes[1],dataframes[0]]

# Concatenate the DataFrames
merged_df = pd.concat(dfs_in_order,ignore_index=True)

# Display the merged DataFrame
merged_df

In [ ]:
merged_df.info()

from the info function displays the new data contains 152 observations across 8 columss from march 24 to september 24.
the first colum shows dates in datetime format, followed by each category of coffee as well as the index values as floats. the added year and month number of each observation is in integer format.


### Rename column names prior to merging both datasets

This will enable to combine previous data from original dataset to have a bigger pool of observations to feed more data in the modeling part. Is expected the final dataset to combine data from feb/23 to sep/24

- df1 = icip_data > contains the original dataset (Feb 2023 - Feb 2024)
- df2 = merged_df > contains the new dataset (Mar 2024 - Sep 2024)



In [ ]:
df1 = icip_data
df2 = merged_df


In [ ]:
df1

In [ ]:
# Ensure that the 'Date' column in both df1 and df2 is in datetime format
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])

# Rename the columns in df2 to match the structure of df1
df2.columns = ['date', 'I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas', 'year', 'month']

# Concatenate df1 and df2 into a single DataFrame
combined_df = pd.concat([df1, df2], ignore_index=True)

# Sort by the 'Date' column to ensure chronological order
combined_df = combined_df.sort_values(by='date').reset_index(drop=True)

# Optionally, save the final DataFrame to a CSV file
#combined_df.to_csv('final_combined_data.csv', index=False)

# Test: print the first few rows to verify the result
print(combined_df.head())

In [ ]:
combined_df

The combined dataset on the correct stucture can help to make better explorations on the next sections. 

# 2. Exploratory Data Analysis


This section aims to see how the data is presented, such as basic statistics and its distribution over time.  They can help to identify average levels, variability, and the range of values 
(large deviations might suggest volatility, which could be important for modeling in later stages of experimentation)


### 2.1 Summary statistics and checking for missing values

From the *info* fucntion we get 431 rows of numerical data, with the first colum showing values in datetime type followed by the ICO composite prices and each category of coffee used for the index. The last 2 columsn are the ones added earlier with the month and year of each data point. 

This analysis will help to interpret basic price dynamics before diving into more complex forecasting techniques, such as SaARIMA and LSTMs



In [ ]:
#basic info

combined_df.info()

In [ ]:
# Summary Statistics for numeric columns 
combined_df.describe()

In [ ]:
# check missing values
print(combined_df.isnull().sum())

In [ ]:
# check missing values

print(combined_df.isna().sum())

## 2.2 Trends over time 

### Plotting trends overtime to begin understanding how this new dataset is presented

This also helps in highlighting any major shifts or trends which are quite pronounced in this dataset. 


### a. Comparing the different categories over time:

Each category has a different weight to calculate the final composite. **(get data on this)**


In [ ]:
# Plot other categories of coffee over time
plt.figure(figsize=(10, 6))
plt.plot(combined_df['date'], combined_df['I-CIP'], label='I-CIP')
plt.plot(combined_df['date'], combined_df['colombian_milds'], label='Colombian Milds')
plt.plot(combined_df['date'], combined_df['other_milds'], label='Other Milds')
plt.plot(combined_df['date'], combined_df['brazilian_nat'], label='Brazilian Naturals')
plt.plot(combined_df['date'], combined_df['robustas'], label='Robustas')
plt.xlabel('Date')
plt.ylabel('Price (US cents/lb)')
plt.title('Coffee Types Trend Over Time')
plt.xticks(rotation=45)
plt.legend()
plt.show()


changing labels for date axis for easier visualisation (ie from 2023-03 to MAR 2023)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Define only the columns you want to plot (excluding the last two columns)
columns_to_plot = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']

plt.figure(figsize=(10, 6))

# Iterate over the selected columns and plot each one
for column in columns_to_plot:
    plt.plot(combined_df['date'], combined_df[column], label=column)

# Customize x-axis to show months (use date format for better readability)
plt.xlabel('Month')
plt.ylabel('Price (US cents/lb)')
plt.title('Price Fluctuations of ICO Composite Indicator and Coffee Groups Over Time')
plt.legend()

# Format the x-axis labels to show the month name with better spacing
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Shows every 3rd month
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

plt.xticks(rotation=45)  # Rotate for better readability

# Show plot
plt.show()

A few things to consider fgrom this main plot:


The plot above illustrates the price fluctuations of the ICO Composite Indicator (I-CIP) and four key coffee categories—Colombian Milds, Other Milds, Brazilian Naturals, and Robustas—over a period from Febuary 2023 to September 2024. 

all categories exhibit a general upward trend, reflecting an overall increase in coffee prices. 

Colombian Milds consistently maintains the highest price, followed by Other Milds, with Brazilian Naturals and Robustas being comparatively lower in price.

The I-CIP line in blue serves as a composite indicator, providing an average of these categories. Notable price peaks appear around March 2023 and mid-2024, suggesting potential seasonal or market-related impacts on prices. Toward the end of the timeline, there is a significant price surge across all categories, which may be attributed to factors such as supply chain disruptions or increased global demand.



###  Setting the date column as index:


In [ ]:
#create new variable for merged_df and reseting date as the index for building time series in later stages
# Set 'date' column as index
combined_df.set_index('date', inplace=True)

#check output
icip_df = combined_df
icip_df.head()

In [ ]:
icip_df.info()

###  b. Checking the range of dataset: 


With the dates as indext we can check the range of the dataset: 

- 607 days however the data collected is at a frequency of BUSINESS DAYS, excluding weekends and holidays, which would account for the difference between the total days (607) and the number of observations (431).

In [ ]:
## Checking how many days are present in the dataset

print(f'Dataframe contains prices between {icip_df.index.min()} {icip_df.index.max()}')
print(f'Total Days = {icip_df.index.max() - icip_df.index.min()} days')

### Once the date is set as index, is possible to measure the range an frequency of data. 




In [ ]:
# making sure the index is set at datetime 
icip_df.index = pd.to_datetime(icip_df.index)

In [ ]:
### From the range, confirm the frequency of the index
print(icip_df.index.freq)

A freq marked as 'None' makes python treat the date as irregular. Manually setting the frquency as Business days since the frequancy is not really defined.
This can have a series of benefits:
- Align  data with time-based operations.
- Perform accurate rolling calculations and time series decomposition.
- Handle missing data systematically.
- Use advanced time series models and resampling.


https://pandas.pydata.org/pandas-docs/version/0.16/timeseries.html

In [ ]:
icip_df = icip_df.asfreq('B')  # B stands for Business Days

In [ ]:
### From the range, confirm the frequency of the index
print(icip_df.index.freq)

In [ ]:
icip_df.info()

After setting the freqeuncy as busines days, the index automatically added 3 more empty rows to the dataset.
index shows **434** entries while non null count is **431**. That means data imputation is required (which will be addressed in further sections for data preparation)


### c. Value distribution across categories

In [ ]:
# Define colors from the Set2 palette
colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']

# Specify the columns you want to include in the boxplot
selected_columns = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']  

# Create boxplot traces
box_traces = []
for i, column in enumerate(selected_columns):
    color = colors[i % len(colors)] 
    box_trace = go.Box(y=icip_df[column], name=column, marker=dict(color=color))
    box_traces.append(box_trace)

# Create layout
layout = go.Layout(title='Boxplot by Column', yaxis=dict(title='Value'), xaxis=dict(title='Variable'))

# Create figure
fig = go.Figure(data=box_traces, layout=layout)

# Show plot
fig.show()

## d. Mean values of each category


In [ ]:

# Bar plot of mean values for each column
plt.figure(figsize=(10, 6))
icip_df[selected_columns].mean().plot(kind='bar', color='skyblue')
plt.title('Mean Values of Each Variable')
plt.xlabel('Variables')
plt.ylabel('Mean')
plt.xticks(rotation=45)
plt.show()

## e.  Correlation Matrix for Coffee Prices

In [ ]:
# Correlation Matrix for Coffee Prices
correlation_matrix = icip_df[['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']].corr()

plt.figure(figsize=(8, 6))
plt.matshow(correlation_matrix, cmap='coolwarm', fignum=1)
plt.colorbar()
plt.title("Correlation Matrix of Coffee Prices")
plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=45)
plt.yticks(range(len(correlation_matrix.columns)), correlation_matrix.columns)
plt.show()

###### plot monthy only by category 

>>The matrix highlights that colombian_milds and other_milds are closely related and likely to be primary influencers on the I-CIP index. In contrast, robustas exhibits more independence from other types. These relationships provide a foundation for selecting variables in a forecasting model, where incorporating highly correlated categories might improve predictive accuracy.<<



###  2.2.1 ICO Composite Indicator Price (I-CIP) 

I-CIP is the main feature to be used for this analysis. 
 
 
There’s a clear upward trend in the I-CIP index from early 2023 to late 2024. This suggests that coffee prices, as represented by I-CIP, have generally increased over this period.

Initial Fluctuations (Early to Mid-2023): In the first part of the timeline (early to mid-2023), the I-CIP prices exhibit noticeable fluctuations, with a few peaks and troughs. This suggests some instability in the market, with prices rising and falling frequently.

In [ ]:
import matplotlib.pyplot as plt

# Plot I-CIP over time
plt.figure(figsize=(10, 6))
plt.plot(icip_df['I-CIP'], label='I-CIP')
plt.xlabel('Date')
plt.ylabel('I-CIP')
plt.title('I-CIP Trend Over Time')
plt.xticks(rotation=45)
plt.legend()
plt.show()

looking closer at the gap in thi line plot:
between dec 23 and jan 24 there seems to have a missing value there


### 2.2.1

### a. Checking monthly seasonality

This plot shows the seasonalityvariation in the composite prices over the years 

In [ ]:
# Extract year and month from the index

## plot only for 2023 and plot a separate for 2024
#are variations in price the same in both year????




icip_df['year'] = icip_df.index.year
icip_df['month'] = icip_df.index.month_name().str[:3]  # This will give  the three-letter month abbreviation.

# Draw Plot
plt.figure(figsize=(12, 7), dpi=80)
sns.boxplot(x='month', y='I-CIP', data=icip_df)

# Set Title
plt.title('Month-wise Box Plot of I-CIP Prices of 2023/2024\n(The Seasonality)', fontsize=18)

# Show the plot
plt.show()

In [ ]:
icip_df.info()

##  b. Separating values montly per each year

Re-arranging the order for yearly plots since the data is not complete, since we have for 2023: Feb to Dec and 2024: Jan to Sep, the labels are done manually below

In [ ]:
# Filter data by year
df_2023 = icip_df[icip_df['year'] == 2023]
df_2024 = icip_df[icip_df['year'] == 2024]

# Plot for 2023
plt.figure(figsize=(12, 7))
sns.boxplot(x='month', y='I-CIP', data=df_2023, order=['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.title('Month-wise Box Plot of I-CIP Prices in 2023\n(The Seasonality)', fontsize=18)
plt.xlabel('Month')
plt.ylabel('I-CIP Price (In US cents/lb)')
plt.show()

# Plot for 2024 (up to September)
plt.figure(figsize=(12, 7))
sns.boxplot(x='month', y='I-CIP', data=df_2024, order=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'])
plt.title('Month-wise Box Plot of I-CIP Prices in 2024\n(The Seasonality)', fontsize=18)
plt.xlabel('Month')
plt.ylabel('I-CIP Price (In US cents/lb)')
plt.show()

## 2.3 Data Cleaning: Checking for Missing Dates 

for determine the right frequency


 The output of the first code below shows exacly how many days contains in the dataset, based on the indexes. while the second one will say how many of those total days are actually the business days containing data for the analisis 


In [ ]:
## Checking how many days are present in the dataset

print(f'Dataframe contains prices between {icip_df.index.min()} {icip_df.index.max()}')
print(f'Total Days = {icip_df.index.max() - icip_df.index.min()} days')

In [ ]:
# Generate a date range for 366 days from the start of the data

start_date = icip_df.index.min()
end_date = icip_df.index.max()  


# Generate a range of business days within this period
business_days = pd.bdate_range(start=start_date, end=end_date)

# Now compare the business_days with icip_df index to find out missing dates
missing_dates = business_days.difference(icip_df.index)

print(f"Total number of expected business days: {len(business_days)}")
print(f"Total number of actual days in data: {icip_df.shape[0]}")
print(f"Total number of missing dates: {len(missing_dates)}")
print("Missing dates are:")
print(missing_dates)

The code above says there are no missing dates in the dataframe, however, it still shows there are a few NaN values. This is confirmed below with the *is null* code that shows a total of 3 missing values across all columns:  out of the 434 entries computed by the datetime index, ranging from 2023-02-01 to 2024-09-30

In [ ]:
# Sum of Nan values
#icip_df.info()
print(icip_df.shape)
print(icip_df.isnull().sum())


**These different methods all confirm there are missing values present in the combined dataset**

In [ ]:
# if there is any missing values will come back TRUE
icip_df.isnull().values.any()

Booleans will say True for any missing valies

In [ ]:
nan_df = icip_df.isna()
print(nan_df)

Separate dataframe isolating the dates with True/False booleans 

In [ ]:
nan_rows = icip_df.isna().any(axis=1)
print(nan_rows)

In [ ]:
#filter rows with nan values
nan_rows = icip_df[icip_df.isna().any(axis=1)]

**Plot figure to highlight missing values**

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(icip_df.isna(), cbar=False, cmap="viridis")
plt.show()

**EXTRACT MISSING DATES FROM THE INDEX**


In [ ]:
# Display the dates with missing data
nan_rows.index

In [ ]:
nan_rows

## Once the  MISSING DATES are identified
## 2.4 DATA IMPUTATION METHODS

MODELS CANT HANDLE MISSING DATA 

**Missing dates**  
- **'2023-12-25'**: December 25th: Christmas Day
- **'2023-12-26'**: December 26th: Often observed as Boxing Day or a Christmas holiday extension
- **'2024-01-01'**: New Year's Day

From the early eda, it was observed no missing values per se, however, for a timeseries analysis, the date in the index must be following the correct sequence ("order?"). it's the main characteristic of modeling this type of data. So eventhough there were no missing values (all dates had values) the date range was incomplete. 

**
Another insight from this is that the composite price is published by the ICO even in regular holidays? 
The price composite is agregated values from US and EU (germany+france), this could be an indicative of why there are only 3 dates not in their database. 

For this project, lets add these three dates by chacking the moving average and the foward fill methods.
There is also an alternatite to model the holiday effect on trend/seasonality of the data by including special events or dummy variables (but for the purpose of this experimentation, it wont be necessary because only 3 dates are a relatively small number of holidays to be considered in modeling. 

## a. DAta interpolation 

This section will compare 3 different methods for interpolation of missing values in the ICIP dataset

- foward fill
- backward fill 
- linear interpolation


In [ ]:
#copy.info()

In [ ]:
# copy of the original DataFrame without 'year' and 'month' columns
copy = icip_df.drop(columns=['year', 'month']).copy()

the last 2 columns were only added to facilite some of the montkly plots, ill copy the main data as a separate dataframe for more statistical measurements

In [ ]:
# Forward Fill
df_ffill = copy.ffill()

# Backward Fill
df_bfill = copy.bfill()

# Linear Interpolation 
df_interpolate = copy.interpolate()

# Get the indices of rows with NaN values in the original DataFrame
nan_indices = copy[copy.isna().any(axis=1)].index

# compare the results for the `I-CIP` column at the NaN rows
print("Forward Fill:")
print(df_ffill['I-CIP'].loc[nan_indices])

print("\nBackward Fill:")
print(df_bfill['I-CIP'].loc[nan_indices])

print("\nInterpolation:")
print(df_interpolate['I-CIP'].loc[nan_indices])

## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html

### b.  Visual comparison of data imputation methods 

In [ ]:
# Create a new DataFrame for comparison to avoid modifying the original 'copy'
compare_df = copy.copy()

# Apply imputation methods to create comparison columns
compare_df['price_forward_fill'] = copy['I-CIP'].ffill()
compare_df['price_backward_fill'] = copy['I-CIP'].bfill()
compare_df['price_interpolate'] = copy['I-CIP'].interpolate()

# Create subplots for visual comparison of imputation methods
fig, axes = plt.subplots(4, 1, sharex=True, figsize=(12, 14))

# Set date format for x-axis
axes[3].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
axes[3].xaxis.set_major_locator(mdates.MonthLocator(interval=2))  # Adjust interval as needed
axes[3].xaxis.set_minor_locator(mdates.MonthLocator())

# Plot the original data with missing points on each subplot for comparison
axes[0].plot(compare_df.index, compare_df['I-CIP'], label='Original Data (with NaNs)', color='blue', linestyle='-', marker='o')
axes[0].set_title('Original Data with Missing Points')
axes[0].legend()

# Forward Fill - plot both original and forward-filled data
axes[1].plot(compare_df.index, compare_df['I-CIP'], label='Original Data (with NaNs)', color='blue', linestyle='-', marker='o')
axes[1].plot(compare_df.index, compare_df['price_forward_fill'], label='Forward Fill', color='orange', linestyle='--', marker='x')
axes[1].set_title('Forward Fill Comparison')
axes[1].legend()

# Backward Fill - plot both original and backward-filled data
axes[2].plot(compare_df.index, compare_df['I-CIP'], label='Original Data (with NaNs)', color='blue', linestyle='-', marker='o')
axes[2].plot(compare_df.index, compare_df['price_backward_fill'], label='Backward Fill', color='red', linestyle='--', marker='+')
axes[2].set_title('Backward Fill Comparison')
axes[2].legend()

# Linear Interpolation - plot both original and interpolated data
axes[3].plot(compare_df.index, compare_df['I-CIP'], label='Original Data (with NaNs)', color='blue', linestyle='-', marker='o')
axes[3].plot(compare_df.index, compare_df['price_interpolate'], label='Linear Interpolation', color='green', linestyle='--', marker='s')
axes[3].set_title('Linear Interpolation Comparison')
axes[3].legend()

# Set labels and rotate date labels
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Price')
plt.setp(axes[3].get_xticklabels(), rotation=45, ha='right')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

Linear interpolation is typically the most appropriate for time series data because it provides a gradual transition between values, which better mimics natural fluctuations in price indices.
Forward and Backward Fill are simpler methods and may be suitable if you believe that missing values should hold steady from a previous or upcoming value, but they can introduce unrealistic flat segments.


https://medium.com/@aaabulkhair/data-imputation-demystified-time-series-data-69bc9c798cb7

https://365datascience.com/tutorials/time-series-analysis-tutorials/pre-process-time-series-data/

### c. Adding linear interpolation to main dataset>

In [ ]:
# Apply linear interpolation to fill missing values in the DataFrame
copy_interpolated = copy.interpolate(method='linear')

# Verify that there are no remaining NaN values
print(copy_interpolated.isna().sum())

# Display a sample of the DataFrame to check the interpolation results
copy_interpolated.head()
#print(copy_interpolated.shape())

## 2.5 EDA with interpolated dataset

## a. boxplot with the interpolated data


In [ ]:



copy_interpolated['year'] = copy_interpolated.index.year
copy_interpolated['month'] = copy_interpolated.index.month_name().str[:3]  # This will give  the three-letter month abbreviation.

# Draw Plot
plt.figure(figsize=(12, 7), dpi=80)
sns.boxplot(x='month', y='I-CIP', data=copy_interpolated)

# Set Title
plt.title('Month-wise Box Plot of I-CIP Prices of 2023/2024\n(The Seasonality)', fontsize=18)

# Show the plot
plt.show()

### b.Pairplot across categories 

The darker points (higher I-CIP values) are more prevalent in 2024, indicating an increase in the I-CIP index, which aligns with the upward trend seen in the scatter plots across coffee types.

There’s a high correlation among colombian_milds, other_milds, brazilian_nat, and robustas, which indicates that changes in the price of one type are closely mirrored by changes in others.

In [ ]:
sns.pairplot(copy_interpolated, hue='I-CIP', diag_kind='hist')

## 2.5.1 Montly prices (boxplot) per year

In [ ]:
# Filter data by year
df_23 = copy_interpolated[copy_interpolated['year'] == 2023]
df_24 = copy_interpolated[copy_interpolated['year'] == 2024]

# Plot for 2023
plt.figure(figsize=(12, 7))
sns.boxplot(x='month', y='I-CIP', data=df_23, order=['Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.title('Month-wise Box Plot of I-CIP Prices in 2023\n(The Seasonality)', fontsize=18)
plt.xlabel('Month')
plt.ylabel('I-CIP Price (In US cents/lb)')
plt.show()

# Plot for 2024 (up to September)
plt.figure(figsize=(12, 7))
sns.boxplot(x='month', y='I-CIP', data=df_24, order=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep'])
plt.title('Month-wise Box Plot of I-CIP Prices in 2024\n(The Seasonality)', fontsize=18)
plt.xlabel('Month')
plt.ylabel('I-CIP Price (In US cents/lb)')
plt.show()

copy_interpolated

#### Plot month-wise price distribution for each cetegory

This structure makes it easy to compare the seasonal trends across different coffee types.


    Colombian Milds consistently have higher prices than the other categories, followed by Other Milds and Brazilian Naturals. Robustas remain at the lowest price range. This order aligns with market expectations, as Colombian and Other Milds are typically *considered higher-quality coffee* (ref) types and therefore command higher prices. Across all categories, prices tend to be lower in the early months (February to May) and then gradually increase throughout the middle of the year, peaking around September.For most categories, there is a noticeable drop in prices from October to January, which could reflect seasonal demand changes or other external market factors affecting coffee prices.

In [ ]:
# Define the list of categories to plot
categories = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']
num_categories = len(categories)

# Create subplots: 1 row for each category
fig, axes = plt.subplots(num_categories, 1, figsize=(12, 14), sharex=True)

# Plot each category in a separate subplot
for i, category in enumerate(categories):
    sns.boxplot(x='month', y=category, data=copy_interpolated, ax=axes[i])
    axes[i].set_title(f'Month-wise Box Plot of {category} Prices (2023/2024)', fontsize=12)
    axes[i].set_ylabel('Price (US cents/lb)')
    
# Set common x-axis label and rotate month labels for readability
axes[-1].set_xlabel('Month')
for ax in axes:
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

# Adjust layout for clarity
plt.tight_layout()
plt.show()

2.5.1

## a. Outliers

In [ ]:
# Filter rows where prices are outliers for each month
# 
q1 = icip_df.groupby('month')['I-CIP'].quantile(0.25)
q3 = icip_df.groupby('month')['I-CIP'].quantile(0.75)
iqr = q3 - q1

# Define outliers as points outside 1.5 * IQR
outliers = icip_df.apply(lambda x: (x['I-CIP'] < q1[x['month']] - 1.5 * iqr[x['month']]) |
                               (x['I-CIP'] > q3[x['month']] + 1.5 * iqr[x['month']]), axis=1)

# Display or analyze the outliers
outlier_data = icip_df[outliers]
outlier_data

In [ ]:

# Plot the I-CIP prices
plt.figure(figsize=(12, 6))
plt.plot(copy_interpolated.index, copy_interpolated['I-CIP'], label='I-CIP Prices', color='blue')
plt.scatter(outlier_data.index, outlier_data['I-CIP'], color='red', label='Outliers')  # Highlight outliers

# Formatting the plot
plt.title('I-CIP Prices with Outliers Highlighted')
plt.xlabel('Date')
plt.ylabel('I-CIP Price (in US cents/lb)')
plt.legend()
plt.show()


b. Explore Correlations with Other Variables


In [ ]:
# Step 1: Select the relevant columns (e.g., I-CIP and other coffee categories) for the outliers
outlier_correlation = outlier_data[['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']]

# Step 2: Calculate correlations
outlier_corr = outlier_correlation.corr()
print(outlier_corr)

### c. Year-over-Year Monthly Comparison:

### Monthly average

This plot aims to compare the avarage of icip prices in 2023 and 2024, to highlight differences in each year.

Overall, the year of 24 is represented by higher average, confirming the upward trend seen in line plots in item c.



In [ ]:
import calendar

In [ ]:
# Set the month order to ensure chronological sorting
month_order = list(calendar.month_abbr[1:])  # ['Jan', 'Feb', ..., 'Dec']
icip_df['month'] = pd.Categorical(icip_df['month'], categories=month_order, ordered=True)

# Monthly average comparison
monthly_avg = icip_df.groupby(['year', 'month'])['I-CIP'].mean().unstack(level=0)
monthly_avg.plot(kind='bar', figsize=(12, 6))
plt.title('Average Monthly I-CIP Prices in 2023 vs 2024')
plt.xlabel('Month')
plt.ylabel('Average I-CIP Price (in US cents/lb)')
plt.show()

months appear in alphabetical order instead of chronological? >>> **adjust\!**

In [ ]:
#Monthly Average Plot
monthly_avg = icip_df['I-CIP'].resample('M').mean()

plt.figure(figsize=(10, 6))
plt.plot(monthly_avg, label="Monthly Average I-CIP")
plt.title("Monthly Average I-CIP Prices")
plt.xlabel("Date")
plt.ylabel("I-CIP Price (In US cents/lb)")
plt.legend()
plt.show()



### d.  Monthly Average I-CIP Prices with Regional Harvest Annotations

In [ ]:
# Set the month order to ensure chronological sorting
#month_order = list(calendar.month_abbr[1:])  # ['Jan', 'Feb', ..., 'Dec']
#icip_df['month'] = pd.Categorical(icip_df['month'], categories=month_order, ordered=True)

# Group by both year and month to keep month names in chronological order
monthly_avg_df = icip_df.groupby([icip_df.index.year, 'month'])['I-CIP'].mean().unstack(level=0)

# Create the plot with annotations of harvest 
plt.figure(figsize=(14, 7))
monthly_avg_df.plot(kind='bar', color=['skyblue', 'salmon'], ax=plt.gca())
plt.xlabel('Month')
plt.ylabel('Average I-CIP Price (in US cents/lb)')
plt.title('Monthly Average I-CIP Prices by Year with Harvest Annotations')

# Annotate months with regional harvests (approximately)
harvest_annotations = {
    'Jan': 'South America, Africa',
    'Feb': 'South America, Africa',
    'Mar': 'South America',
    'Apr': 'Central America, South America',
    'May': 'Asia',
    'Jun': 'South America, Africa, Asia',
    'Jul': 'Asia, Africa',
    'Aug': 'Asia, Africa',
    'Sep': 'Asia',
    'Oct': 'South America, Africa, Asia',
    'Nov': 'Central America, Africa',
    'Dec': 'South America, Africa'
}

# Add annotations at 45-degree angle for readability
for month_idx, (month, regions) in enumerate(harvest_annotations.items()):
    plt.text(month_idx - 0.15, monthly_avg_df.loc[month].max() + 5, 
             regions, ha='center', rotation=45, color='black', fontsize=8)

# Adjust x-axis labels
plt.xticks(rotation=45)
plt.legend(title="Year", loc="upper right")
plt.show()



# harverst dates extracted from 
#Source: https://coffeehunter.com/coffee-seasonality/ accessed on 30/10
# https://www.fairmountaincoffee.com/category-s/102.htm accessedon 30/10

#### Heatmap of Monthly Price Averages 

The heatmat below aims to identify months where prices tend to dip or spike, then cross-reference with known harvest periods. The color intensity provides a quick overview of price levels each month.

In [ ]:
# Create a DataFrame for monthly averages
monthly_avg_df = icip_df.groupby([icip_df.index.year, icip_df.index.month])['I-CIP'].mean().unstack()

plt.figure(figsize=(12, 6))
sns.heatmap(monthly_avg_df, annot=True, cmap="YlGnBu", fmt=".1f", linewidths=0.5)
plt.title('Monthly I-CIP Price Averages (in US cents/lb)')
plt.xlabel('Month')
plt.ylabel('Year')
plt.show()

In [ ]:
#new_df.index

In [ ]:
new_df = copy_interpolated
new_df.info()

## e. Monthly variation (%) of for each category
### ICIP and its components 


In [ ]:
# Resample data to monthly frequency and calculate percentage change
monthly_variations = new_df[['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']].resample('M').ffill().pct_change() * 100

# Set up the figure and subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 12), sharex=True)
fig.suptitle("Monthly Variation (%) of Coffee Prices by Category", fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# List of categories to plot
categories = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']

# Plot each category's monthly variation in its own subplot
for i, category in enumerate(categories):
    axes[i].plot(monthly_variations.index, monthly_variations[category], label=f'{category} Monthly Variation (%)')
    axes[i].axhline(0, color='gray', linestyle='--')  # Add a horizontal line at 0
    axes[i].set_title(f"{category} Monthly Variation")
    axes[i].set_ylabel("Monthly Variation (%)")
    axes[i].legend()

# Hide any unused subplots
for j in range(len(categories), len(axes)):
    axes[j].axis('off')

# Adjust layout and show the plot
plt.xlabel("Date")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Leave space for the main title
plt.show()

## 2.5.2 Volatility
## a. Weekly variations


Resampling the data to weekly frequency and calculate the percentage change from one week to the next

In [ ]:
# Calculate weekly percentage change (variation) for each category
weekly_variations = new_df[['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']].resample('W').ffill().pct_change() * 100

# Plot each category's weekly variation in separate subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 12), sharex=True)
fig.suptitle("Weekly Variation (%) of Coffee Prices by Category", fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# List of categories to plot
categories = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']

# Plot each category's weekly variation
for i, category in enumerate(categories):
    axes[i].plot(weekly_variations.index, weekly_variations[category], label=f'{category} Weekly Variation (%)')
    axes[i].axhline(0, color='gray', linestyle='--')  # Add a horizontal line at 0
    axes[i].set_title(f"{category} Weekly Variation")
    axes[i].set_ylabel("Weekly Variation (%)")
    axes[i].legend()

# Hide any unused subplots
for j in range(len(categories), len(axes)):
    axes[j].axis('off')

# Adjust layout and show the plot
plt.xlabel("Date")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Leave space for the main title
plt.show()

## b. Daily variations


In [ ]:
# Calculate daily percentage change (variation) for each category
daily_variations = new_df[['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']].pct_change() * 100

# Plot each category's daily variation in separate subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 12), sharex=True)
fig.suptitle("Daily Variation (%) of Coffee Prices by Category", fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each category's daily variation
for i, category in enumerate(categories):
    axes[i].plot(daily_variations.index, daily_variations[category], label=f'{category} Daily Variation (%)')
    axes[i].axhline(0, color='gray', linestyle='--')  # Add a horizontal line at 0
    axes[i].set_title(f"{category} Daily Variation")
    axes[i].set_ylabel("Daily Variation (%)")
    axes[i].legend()

# Hide any unused subplots
for j in range(len(categories), len(axes)):
    axes[j].axis('off')

# Adjust layout and show the plot
plt.xlabel("Date")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Leave space for the main title
plt.show()

## c. Highlight the high volatility periods

## Weekly and daily

Based on industry standards, as cited by Lordemann et al(2021) 

- Weekly Volatility Threshold: Flag weeks with variations above ±5%.
Weekly Threshold: Weekly changes greater than ±5% are often used in commodity markets to flag significant shifts, particularly for goods with high seasonality or sensitivity to external events.

- Daily Volatility Threshold: Flag days with variations above ±3%.
In financial markets, daily price changes greater than ±3% are often considered high volatility. Since coffee prices can be sensitive to market and environmental factors, this threshold could be reasonable.

Source: https://repositorio.cepal.org/server/api/core/bitstreams/ca31a1a5-c246-49e4-81e0-be62a16f3c26/content

#### identify treshholds
As an a dditional justification for the tresholds above, the code below shows a percentile based approach to compare the industry standards (+-3% and +-5%) to the actual extreme variations in the ICIP data, which falls close to the benchmark numbers. 
With a daily treshold of 2.72% and weekly of 6.53%

In [ ]:
##### Calculate percentiles for daily and weekly variations
daily_threshold = daily_variations['I-CIP'].quantile(0.95)
weekly_threshold = weekly_variations['I-CIP'].quantile(0.95)

print(f"95th Percentile-Based Daily Threshold: ±{daily_threshold:.2f}%")
print(f"95th Percentile-Based Weekly Threshold: ±{weekly_threshold:.2f}%")

#### Define most volatile days and weeks based on threshold calculated above

In [ ]:
# Define thresholds based on daily/weekly variations calculated above
weekly_threshold = 6.53  # Weekly variation above +/- 6.53% considered volatile
daily_threshold = 2.72   # Daily variation above +/-2.72 considered volatile 

# Find volatile weeks
volatile_weeks = weekly_variations[(weekly_variations > weekly_threshold) | (weekly_variations < -weekly_threshold)].dropna(how='all')
print("Volatile Weeks:")
print(volatile_weeks)

# Find volatile days
volatile_days = daily_variations[(daily_variations > daily_threshold) | (daily_variations < -daily_threshold)].dropna(how='all')
print("Volatile Days:")
print(volatile_days)

## Visualise Volatile periods




In [ ]:


# Plot daily variation with volatile days highlighted
plt.figure(figsize=(12, 6))
plt.plot(daily_variations.index, daily_variations['I-CIP'], label='I-CIP Daily Variation')
plt.scatter(volatile_days.index, volatile_days['I-CIP'], color='red', label='Volatile Days', marker='o')
plt.axhline(y=0, color='gray', linestyle='--')
plt.title("I-CIP Daily Variation with Volatile Days Highlighted")
plt.xlabel("Date")
plt.ylabel("Daily Variation (%)")
plt.legend()
plt.show()

In [ ]:
# plot for weekly variations
plt.figure(figsize=(12, 6))
plt.plot(weekly_variations.index, weekly_variations['I-CIP'], label='I-CIP Weekly Variation')
plt.scatter(volatile_weeks.index, volatile_weeks['I-CIP'], color='red', label='Volatile Weeks', marker='o')
plt.axhline(y=0, color='gray', linestyle='--')
plt.title("I-CIP Weekly Variation with Volatile Weeks Highlighted")
plt.xlabel("Date")
plt.ylabel("Weekly Variation (%)")
plt.legend()
plt.show()

In [ ]:
daily_variations.head()

In [ ]:
# Calculate rolling standard deviation (e.g., 7-day) for each category
rolling_volatility = daily_variations['I-CIP'].rolling(window=7).std()

# Plot the rolling volatility
plt.figure(figsize=(12, 6))
plt.plot(rolling_volatility.index, rolling_volatility, label="7-Day Rolling Volatility", color='blue')
plt.axhline(y=rolling_volatility.mean(), color='red', linestyle='--', label="Mean Volatility")
plt.title("7-Day Rolling Volatility for I-CIP")
plt.xlabel("Date")
plt.ylabel("Rolling Volatility (%)")
plt.legend()
plt.show()

In [ ]:
#pip install calmap

In [ ]:
import calmap

# Resample to daily absolute values of I-CIP volatility for visualization
daily_volatility_icip = daily_variations['I-CIP'].abs()

# Plot calendar heatmap for daily volatility
plt.figure(figsize=(14, 8))
calmap.calendarplot(daily_volatility_icip, cmap='Reds', fillcolor='lightgrey', linewidth=0.1)
plt.title("Calendar Heatmap of Daily Volatility for I-CIP")
plt.show()

 Rolling Volatility (Standard Deviation)
Another approach for pinpointing volatility through a sliding window using a rolling standard deviation.
The code belows attemp tto do it


In [ ]:
# Calculate 4-week rolling standard deviation for each category
weekly_rolling_volatility = weekly_variations.rolling(window=4).std()

# Identify periods where rolling volatility exceeds a chosen threshold
high_weekly_volatility = weekly_rolling_volatility[weekly_rolling_volatility > weekly_threshold]
print("High Weekly Volatility Periods:")
print(high_weekly_volatility.dropna(how='all'))

In [ ]:
# Calculate 7-day rolling standard deviation for each category
daily_rolling_volatility = daily_variations.rolling(window=7).std()

# Identify periods where rolling volatility exceeds a chosen threshold
high_daily_volatility = daily_rolling_volatility[daily_rolling_volatility > daily_threshold]
print("High Daily Volatility Periods:")
print(high_daily_volatility.dropna(how='all'))

## d. Checking periods of high volatility across all variables?

## Daily variation

In [ ]:
# Define your volatility threshold for daily variations
daily_threshold = 2.72

# Find volatile days for each category
daily_volatile_days = daily_variations[(daily_variations > daily_threshold) | (daily_variations < -daily_threshold)]

# Set up subplots for daily variations
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 12), sharex=True)
fig.suptitle("Daily Variation (%) of Coffee Prices by Category with Volatile Days Highlighted", fontsize=16)

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot each category's daily variation with volatile days highlighted
for i, category in enumerate(categories):
    axes[i].plot(daily_variations.index, daily_variations[category], label=f'{category} Daily Variation (%)')
    axes[i].scatter(daily_volatile_days.index, daily_volatile_days[category], color='red', label='Volatile Days', marker='o')
    axes[i].axhline(0, color='gray', linestyle='--')
    axes[i].set_title(f"{category} Daily Variation")
    axes[i].set_ylabel("Daily Variation (%)")
    axes[i].legend()

# Hide any unused subplots if there are extra spots
for j in range(len(categories), len(axes)):
    axes[j].axis('off')

# Adjust layout and show the plot
plt.xlabel("Date")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## Weekly variation

In [ ]:
# Assuming categories is the list of your column names, like:
categories = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']

# Define volatility threshold for weekly variations
weekly_threshold = 6.5

# Find volatile weeks for each category
weekly_volatile_weeks = weekly_variations[(weekly_variations > weekly_threshold) | (weekly_variations < -weekly_threshold)]

# Set up subplots for weekly variations
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(15, 12), sharex=True)
fig.suptitle("Weekly Variation (%) of Coffee Prices by Category with Volatile Weeks Highlighted", fontsize=16)

# Flatten axes array for easier indexing
axes = axes.flatten()

# Plot each category's weekly variation with volatile weeks highlighted
for i, category in enumerate(categories):
    axes[i].plot(weekly_variations.index, weekly_variations[category], label=f'{category} Weekly Variation (%)')
    axes[i].scatter(weekly_volatile_weeks.index, weekly_volatile_weeks[category], color='red', label='Volatile Weeks', marker='o')
    axes[i].axhline(0, color='gray', linestyle='--')
    axes[i].set_title(f"{category} Weekly Variation")
    axes[i].set_ylabel("Weekly Variation (%)")
    axes[i].legend()

# Hide any unused subplots if there are extra spots
for j in range(len(categories), len(axes)):
    axes[j].axis('off')

# Adjust layout and show the plot
plt.xlabel("Date")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
#pip install geopandas

### Distribution of volatile Months and Weekdays

In [ ]:
# Add 'Month' and 'Weekday' columns to the DataFrame
daily_variations['Month'] = daily_variations.index.month
daily_variations['Weekday'] = daily_variations.index.day_name()

# Box plot by month
plt.figure(figsize=(12, 6))
sns.boxplot(x='Month', y='I-CIP', data=daily_variations, color='lightblue')
plt.title("Monthly Distribution of Daily Volatility for I-CIP")
plt.xlabel("Month")
plt.ylabel("Daily Variation (%)")
plt.show()

# Box plot by day of the week
plt.figure(figsize=(12, 6))
sns.boxplot(x='Weekday', y='I-CIP', data=daily_variations, color='lightgreen', order=[
            'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title("Weekly Distribution of Daily Volatility for I-CIP")
plt.xlabel("Day of the Week")
plt.ylabel("Daily Variation (%)")
plt.show()


In [ ]:
daily_variations.head()

In [ ]:

#sns.pairplot(daily_variations)

## 2.6 Distribuition

In [ ]:
new_df.info()

In [ ]:
# Changing some of the aesthetics... adding in a kernel desnity estimate:
ax = sns.histplot(new_df['I-CIP'], kde=True, color ='green')
ax.set(xlabel='i-cip',
       ylabel='Frequency',
       title ="ICO Composite Prices  Histogram")

From the histogram above, we can see the data doensnt show a normal distribution of values, the graph below a kde is added to the plot to make it even clearer. The Bimodal distribution indicates the presence of two distinct groups within the dataset, each with its own central tendency. (two populations in the sample)


This often suggests that this dataset may be influenced by two different underlying processes or conditions. In terms of comodity markets, for example, a distribution like this one might reflect different market dynamics or consumption patterns across two separate seasons or market segments. (**find ref**) 

The next step is to consider whether these modes represent meaningful segments that require additional analysis or modeling strategies. By fitting a Gaussian Mixture Model or exploring the data's skewness and kurtosis, we can dive deeper into understanding the characteristics of these subgroups, which can give more insight son how to proceed to tailoring the forecasting models.

In [ ]:
# Create histogram
fig = px.histogram(new_df['I-CIP'], x='I-CIP', nbins=30) #changing the number of bins makes the curve clearer, 10, 20 were also used

# Update layout
fig.update_layout(
    title='I-CIP Histogram',
    xaxis_title='Price',
    yaxis_title='Frequency'
)

# Show plot
fig.show()

From the histogram above, we can see the data doensnt show a normal distribution of values, the graph below a kde is added to the plot to make it even clearer. The Bimodal distribution indicates the presence of two distinct groups within the dataset, each with its own central tendency. (two populations in the sample)


This often suggests that this dataset may be influenced by two different underlying processes or conditions. In terms of comodity markets, for example, a distribution like this one might reflect different market dynamics or consumption patterns across two separate seasons or market segments.
The next step is to consider whether these modes represent meaningful segments that require additional analysis or modeling strategies. By fitting a Gaussian Mixture Model or exploring the data's skewness and kurtosis, we can dive deeper into understanding the characteristics of these subgroups, which can give more insight son how to proceed to tailoring the forecasting models.

In [ ]:

# Create a figure with two subplots to check for distribution
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot histogram of icip prices in the first subplot
axs[0].hist(copy['I-CIP'], bins=10)
axs[0].set_title('Histogram of I-CIP values')
axs[0].set_xlabel('Value')
axs[0].set_ylabel('Frequency')

# Plot probability plot of prices in the second subplot
stats.probplot(copy['I-CIP'], dist='norm', plot=axs[1])
axs[1].set_title('Probability Plot of Values')

# Adjust the layout to avoid overlapping titles and labels
plt.tight_layout()

# Show the plots
plt.show()

the data shows some characteristics of a normal distribution but with potential signs of bimodality and the presence of outliers or heavy tails. This can often happen in real-world data where multiple factors contribute to the observed values, which can lead to more complex distributions deviating from the ideally normal dist.

### checking skewness of data

from the plot below we see the data is presenting a bimodal distribuition, which indicates the non normality of values. 

The lack of simmetry in distribution (tail extending to the right side) is confirmed by printing the actual skeweness measure which is 0.889. 

Another statistical measure used was the kurtosis to identify potential outliers in the dataset, represented by the flatness of the tail at a 0.30 

In [ ]:

from scipy.stats import skew, kurtosis
from sklearn.mixture import GaussianMixture

# Visual Inspection
plt.hist(new_df['I-CIP'], bins=30, alpha=0.5, color='blue', density=True)
new_df['I-CIP'].plot(kind='density', color='green')
plt.title('Histogram and Density Plot')
plt.show()

# Check skewness and kurtosis
data_skewness = skew(new_df['I-CIP'])
data_kurtosis = kurtosis(new_df['I-CIP'])
print(f"Skewness: {data_skewness}")
print(f"Kurtosis: {data_kurtosis}")

# Fitting a Gaussian Mixture Model >>> cant fit this model when there are Nan values in the datatset
gmm = GaussianMixture(n_components=2, random_state=0).fit(np.expand_dims(new_df['I-CIP'], 1))
gmm_scores = gmm.score_samples(np.expand_dims(new_df['I-CIP'], 1))

print(f"GMM Converged: {gmm.converged_}") # Check if the algorithm has converged
print(f"GMM Weights: {gmm.weights_}") # Gives the weights of the two components

# Plotting the GMM results
plt.hist(new_df['I-CIP'], bins=30, alpha=0.5, color='blue', density=True)
plt.plot(np.linspace(min(new_df['I-CIP']), max(new_df['I-CIP']), len(gmm_scores)), np.exp(gmm_scores), color='red')
plt.title('Histogram with GMM-estimated Density')
plt.show()

#Source: https://builtin.com/articles/gaussian-mixture-model


**ValueError: Input X contains NaN.** 
GaussianMixture does not accept missing values encoded as NaN natively.

### Weighted average


ICO Composite Indicator Price (I-CIP) is calculated using a weighted average of four coffee groups: Colombian Milds, Other Milds, Brazilian Naturals, and Robustas, with each group contributing a specific weight to the calculation:

- Colombian Milds: 12%
- Other Milds: 21%
- Brazilian Naturals: 30%
- Robustas: 37%


In [ ]:
#print(new_df.info())

In [ ]:
# Calculate the weighted I-CIP
new_df['weighted_I-CIP'] = (0.12 * new_df['colombian_milds'] +
                                 0.21 * new_df['other_milds'] +
                                 0.30 * new_df['brazilian_nat'] +
                                 0.37 * new_df['robustas'])

# Compare weighted I-CIP with actual I-CIP
plt.figure(figsize=(10, 6))
plt.plot(new_df.index, copy_interpolated['I-CIP'], label='Actual I-CIP')
plt.plot(new_df.index, copy_interpolated['weighted_I-CIP'], label='Weighted I-CIP', linestyle='--')
plt.xlabel('Date')
plt.ylabel('I-CIP')
plt.title('Actual I-CIP vs Weighted I-CIP')
plt.legend()
plt.show()

#### Lag plots for I-CIP values
understanding the entropy of icip prices and how correlated they are

the scatter plot bellow shows the relationship between observations and their lags.
"as the lag increases, the correlation between the time series and its lags generally decreases."

Some sort of autocorrelation in the data is visible in lag 1, (t+1). A strong linear relationship indicates a high correlation between an observation and its immediate predecessor. a similar pattern is observed in lag2, with a few datapoints begining to get apart. Lags 3 and 4 are already more spreaded, meaning the correlation between values is also decreasing as the interval between lags grow.

In [ ]:
from pandas.plotting import lag_plot
plt.rcParams.update({'ytick.left' : False, 'axes.titlepad':10})

lp = new_df['I-CIP']

# Plot
fig, axes = plt.subplots(1, 4, figsize=(10,3), sharex=True, sharey=True, dpi=100)
for i, ax in enumerate(axes.flatten()[:4]):
    lag_plot(lp, lag=i+1, ax=ax, c='firebrick')
    ax.set_title('Lag ' + str(i+1))

    
fig.suptitle('Lag Plots of I-CIP prices \n(Points get wide and scattered with increasing lag -> lesser correlation)\n', y=1.15)    

plt.show

The lag plots show the relationship between the current value of the I-CIP prices 
𝑦
(
𝑡
)
y(t) and its values at different time lags 
𝑦
(
𝑡
+
𝑘
)
y(t+k) for 
𝑘
=
1
,
2
,
3
,
k=1,2,3, and 
4
4. Here’s an analysis of each lag plot:

Lag 1:

The points are tightly clustered along a diagonal line, indicating a strong positive correlation between 
𝑦
(
𝑡
)
y(t) and 
𝑦
(
𝑡
+
1
)
y(t+1).
This suggests that there is a strong dependency between consecutive observations, meaning today’s price has a strong relationship with tomorrow’s price.
 the lag plots indicate that the I-CIP series has a strong immediate dependency (Lag 1) that diminishes quickly, supporting a model structure with low-order lags. 

In [ ]:
# Define the number of lags for 1 month
number_of_lags = 21

# Create subplots with 3 columns
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(15, 20))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Generate a lag plot for each lag
for i in range(1, number_of_lags + 1):
    lag_plot(icip_df['I-CIP'], lag=i, ax=axes[i-1])
    axes[i-1].set_title(f'Lag {i}')

# Adjust layout to prevent overlapping
plt.tight_layout()
plt.show()

### Rolling average / Rolling standard deviation


Rolling Mean: The rolling mean is the average of the previous observation window, where the window consists of a series of values from the time series data. Computing the mean for each ordered window. This can significantly help minimize noise in time series data.

In [ ]:
## Rolling Statistics at different periods
window_sizes = [5, 21, 63]  # A week, a month, a quarter,  (approximately)
data_rolling = new_df['I-CIP']  

for window in window_sizes:
    rolling_mean = new_df['I-CIP'].rolling(window=window).mean()
    rolling_std = new_df['I-CIP'].rolling(window=window).std()
    
    plt.figure(figsize=(14, 5))
    plt.plot(new_df['I-CIP'].index, new_df['I-CIP'], label='Original')
    plt.plot(rolling_mean.index, rolling_mean, label=f'Rolling Mean (window={window})')
    plt.plot(rolling_std.index, rolling_std, label=f'Rolling Std Dev (window={window})')
    plt.title(f'Rolling Mean and Standard Deviation (window size = {window})')
    plt.legend()
    plt.show()

## 2.7 Checking for Stationarity

### Hypotesis testing: 

### Augmented Dickey-Fuller test

ADF test is conducted with the following assumptions:

- Null Hypothesis (HO): Series is non-stationary, or series has a unit root.
- Alternate Hypothesis(HA): Series is stationary, or series has no unit root.
If the null hypothesis is failed to be rejected, this test may provide evidence that the series is non-stationary. 
(https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/)

In [ ]:
# ADF Test
# Function to print out results
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    
    
# Call the function and run the test

adf_test(new_df['I-CIP'])

The results of the ADF test indicate that the I-CIP series is non-stationary, as the standard to be considered is a  p value bewlow 0.05

**p-value: 0.99** – p-value is quite high which means  we fail to reject the null hypothesis of the ADF test since is fallsabove the significance level. 

This suggests that the series has a unit root => **non-stationary**.

 [The code for both tests were extracted from Kumar (2023)
https://www.analyticsvidhya.com/blog/2021/06/statistical-tests-to-check-stationarity-in-time-series-part-1/]


Although the adf might be more comon for hypotesis testing, the kpps usualy acts as a complementary evidence for this section. 


### KPPS test for Stationarity

The interpretation of the KPSS test is the opposite to the Dickey-Fuller test:

If the p-value is less than the significance level *(p < 0.05)*, then we reject the null hypothesis, indicating that the time series is not trend-stationary.
If the p-value is greater than the significance level, we fail to reject the null hypothesis, suggesting that the series may be trend-stationary.


the **p-value = 0.01** is less than 0.05, indicating statistical significance. Therefore, we reject the null hypothesis of trend-stationarity. This suggests that the time series may not be trend-stationary. As discussed by Kumar (2023), if both tests conclude the the given series is non-stationary, there is a high indication thart the series is in fact non-stationary. Which is commonly the case for stock prices.

In [ ]:
# Function to print out results in customised manner
from statsmodels.tsa.stattools import kpss
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c', nlags="auto")
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','#Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
    print (kpss_output)

# Call the function and run the test

kpss_test(new_df['I-CIP'])

Since both the ADF and KPSS tests indicate that the I-CIP series is non-stationary:

- The ADF test failed to reject the null hypothesis of non-stationarity (high p-value 0.99).
- The KPSS test rejected the null hypothesis of stationarity (low p-value 0.01).


In an ideal world i would go back and test the data with the holidays and add more years back to further the historical analysis. At this stage, with the time left to compolete the project this methods above will be kept in a drawer for another moment.

- Standardization/Normalization: Use techniques like z-score standardization or min-max scaling to scale the coffee prices to a common range. This helps to handle features with different scales. It removes the mean and scales the data to have unit variance, which can make it easier for some models to learn the patterns in the data.

- Differencing: Compute differences between consecutive stock prices to remove trends and make the data stationary, if necessary, which is often required for time series forecasting or machine learning tasks.

Source: Binita Kumari and Tripti Swarnkar 
    International Journal of Advanced Technology and Engineering Exploration, Vol 8(83)1339 
    
Apply time series analysis techniques beyond autocorrelation and decomposition, as they will be done in further section, and investigate moving averages,to identify further seasonal components, trends, and other patterns in the data. This could also be used as additional features for modeling\

### ADF/KPPS tests for all categories:

First cvode bvelow will renew the trend visualisation across categories

The second code performs the adf and kpps tests on all categories to confirm a similar pattern as observed on previous section of the project.


In [ ]:
# Plot each category to visually inspect trends
categories = ['I-CIP', 'colombian_milds', 'other_milds', 'brazilian_nat', 'robustas']

for category in categories:
    new_df[category].plot(title=f'Time Series of {category}', figsize=(10, 5))
    plt.show()

The idea behind the unit test is that it finds out how
strongly a time series is determined by a trend.

- Null Hypothesis (H0): The time series can be represented by a unit root that is not stationary.
- Alternative Hypothesis (H1): The time series is stationary.

<br>
Interpretation of the p-value

- P-value > 0.05: Accepts the Null Hypothesis
- P-value < 0.05: Rejects the Null Hypothesis

P. Yamak et al (2019) says it's common to use both tests to test fpr stationarity, and in this case, the ADF test provides more convincing evidence of stationarity compared to the KPSS test.

In [ ]:
for category in categories:
    print(f"--- Stationarity Test for {category} ---")
    # ADF Test
    adf_result = adfuller(new_df[category].dropna())
    print('ADF Statistic:', adf_result[0])
    print('p-value:', adf_result[1])

    # KPSS Test
    kpss_result = kpss(new_df[category].dropna(), regression='c')
    print('KPSS Statistic:', kpss_result[0])
    print('p-value:', kpss_result[1])
    print("\n")


## 2.8 Decomposing data for Seasonality and Trend


"Predicting future values is easier with a stationary series, enhancing the precision of statistical models. Thus, for effective predictions, ensuring stationarity is crucial." https://www.analyticsvidhya.com/blog/2018/09/an-introduction-to-non-stationary-time-series-in-python/

The first part of the next code will consists of a lineplot to visualize the time series data of how the values change over time. Next is performed a seasonal decomposition for each column in the DataFrame. It iterates over each column, decomposes the time series into trend, seasonal, and residual/noise components using the seasonal decompose function from statsmodels, and plots the decomposed components. This helps in understanding the underlying patterns and trends in the data.

"Decomposition provides a useful abstract model for thinking about time series generally and for better understanding problems during time series analysis and forecasting." Brownlee,2020 https://machinelearningmastery.com/decompose-time-series-data-trend-seasonality/

"Decomposition is the process of breaking down a time series into its constituent components, typically trend, seasonality, and residual (noise). Trend represents the long-term movement or directionality of the data, seasonality captures periodic fluctuations, and residual represents random fluctuations or noise. Decomposition helps understanding the underlying patterns in the data before modeling and forecasting" Dey, 2024 (https://medium.com/@roshmitadey/time-series-decomposition-62cbf31ab65e) Bonaros, 2021 (https://towardsdatascience.com/time-series-decomposition-in-python-8acac385a5b2)

Finally, the last part of the code returns the autocorrelation function (ACF) and partial autocorrelation function (PACF) plots for each column in the DataFrame. Each plot gives us insights about the autocorrelation and partial autocorrelation at different lags, to help identify the order of autoregressive (AR) and moving average (MA) terms for time series modeling.

Other ways to decompose data for timeseries models, (advanced tecniques for more complex, irregular or non-stationary data)

- Seasonal and Trend decomposition using Loess (STL)
- Wavelet Decomposition
- Singular Spectrum Analysis (SSA)
(Dey, 2024)

In [ ]:
# Seasonal decompositions with different periods.

periods = [63, 21, 5]  # Quartely, Monthly and Weekly (considering business days)
# Function to generate the plots for all periods.
for period in periods:
    decompositions = seasonal_decompose(new_df['I-CIP'], model='additive', period=period)

    # Plotting the components of the decomposition
    plt.rcParams.update({'figure.figsize': (8,8)})
    print(f"Seasonal Decomposition with Period = {period}")
    decompositions.plot()
    plt.show()

From a quick inspection, we can see the a comparison between the raw timeseries data for ICIP prices over the 20m, as well as its fluctuations and an overall trend. 

**TREND** 

In [ ]:
# Initially had the seasonal deco and ACF of all columns gathered in the same code, 
#ended up being too crowded/poluted and hard to comprehend. They are marked as comments bellow. 

# Visualize how values change over time
new_df['I-CIP'].plot(figsize=(12, 6))
plt.title('Time Series Data')
plt.xlabel('Date')
plt.ylabel('Value')
plt.show()

    
# Seasonal decomposition
for category in categories:
    result = seasonal_decompose(new_df[category], model='additive', period=21)  # Assuming a monthly pattern business days
    result.plot()
    plt.suptitle(f'Seasonal Decomposition of {category}')
    plt.tight_layout() 
    plt.show()



# ACF and PACF plots
for category in categories:
    fig, ax = plt.subplots(2, 1, figsize=(10, 6))
    plot_acf(new_df[category], ax=ax[0], lags=40)
    plot_pacf(new_df[category], ax=ax[1], lags=40)
    ax[0].set_title(f'ACF and PACF for {column}')
    plt.tight_layout()  # adjust subplot parameters
    plt.show()

### Decomposition of I-CIP with a period = 5 (Freq as Business days)

In [ ]:
result = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=5) #weekly

# Plot the original and decomposed components
plt.figure(figsize=(12, 8))




plt.subplot(4, 1, 1)
plt.plot(new_df['I-CIP'], label='Original')
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(result.trend, label='Trend')
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(result.seasonal, label='Seasonal')
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(result.resid, label='Residual')
plt.legend()

plt.suptitle( 'Decomposition of I-CIP values period = 5')
plt.tight_layout()
plt.show()

### Decomposition of I-CIP with a period of 21 (montly business days)

 from the seasonal section of graph, it  seems to be present a **regular fluctuation in coffee prices with peaks and troughs that repeat every 21 days** 
 
 in terms of residuals,  if there are significant patterns in the residuals, it could indicate that some aspects of seasonality or trend have not been fully captured by the model. (residulas: the more random the "better" = more white noise



"Residual: These are the irregularities or 'noise' that cannot be attributed to the trend or seasonality. It represents the randomness or irregular occurrences in the data after removing the trend and seasonal components. Ideally, the residuals should not show any pattern (be white noise) if the model has captured all the systematic information in the data." https://towardsdatascience.com/time-series-decomposition-8f39432f78f9#:~:text=Compute%20the%20seasonal%20component%2C%20S,%2F(TR)%20for%20multiplicative%20model.




In [ ]:
result = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=21) #montly BD

# Plot the original and decomposed components
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(new_df['I-CIP'], label='Original')
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(result.trend, label='Trend')
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(result.seasonal, label='Seasonal')
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(result.resid, label='Residual')
plt.legend()


plt.suptitle( 'Decomposition of I-CIP values period = 21')
plt.tight_layout()
plt.show()

In [ ]:
result = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=63) # Quarterly

# Plot the original and decomposed components
plt.figure(figsize=(12, 8))

plt.subplot(4, 1, 1)
plt.plot(new_df['I-CIP'], label='Original')
plt.legend()

plt.subplot(4, 1, 2)
plt.plot(result.trend, label='Trend')
plt.legend()

plt.subplot(4, 1, 3)
plt.plot(result.seasonal, label='Seasonal')
plt.legend()

plt.subplot(4, 1, 4)
plt.plot(result.resid, label='Residual')
plt.legend()


plt.suptitle( 'Decomposition of I-CIP values period = 21')
plt.tight_layout()
plt.show()

## AUTOCORRELATION -  ACF PACF plots 

https://itsudit.medium.com/deciphering-acf-and-pacf-plots-a-guide-to-time-series-forecasting-3323948935fb#:~:text=To%20interpret%20ACF%20and%20PACF,a%20trend%20in%20the%20data.

In [ ]:
# Plot the data
#new_df['I-CIP'].plot(figsize=(14, 7))
#plt.title('Time Series Data')
#plt.show()

# Perform a seasonal decomposition to infer potential seasonality
result = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=21)  #
result.seasonal.plot(figsize=(14, 7))
plt.title('Seasonal Decomposition - Seasonality Component')
plt.show()

# ACF and PACF plots
plot_acf(new_df['I-CIP'].dropna(), lags=40)  # 
plt.title('Autocorrelation Function')
plt.show()

plot_pacf(new_df['I-CIP'].dropna(), lags=40, method='ywm')  # method='ywm' to avoid statistical issues
plt.title('Partial Autocorrelation Function')
plt.show()

###  autocorrelation

- The distance between peaks or troughs can give you an indication of the length of the seasonal cycle.
- The gradual decline from 1 at lag 0 indicates how the correlation between the values decreases as the lags increase
- The significant negative autocorrelation at higher lags could be due to the ??? complex seasonal pattern? 


As a complement to the plot above to reiterate the lack of a defined trend, but the high inlfuence of seasonality. ?

In [ ]:
from pandas.plotting import autocorrelation_plot

# Draw Plot
plt.rcParams.update({'figure.figsize':(9,5), 'figure.dpi':120})
autocorrelation_plot(new_df['I-CIP'].tolist())

In [ ]:

# using daily data and interested in montly seasonality
decomp = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=63)

# Plotting the seasonal component
decomp.seasonal.plot()
plt.title('Seasonal Decomposition - Seasonality Component')
plt.show()

## Fourier Analysis (identify dominant cycles, if any):

The Fourier transform can help remove noise by transforming time series data into the frequency domain, allowing  to filter out noise frequencies. Then, the Fourier inverse transform is applied to obtain the time series after filtering
https://medium.com/@tubelwj/guide-to-time-series-data-pre-processing-methods-0a6df7ee054f



In [ ]:
# Perform a Fourier transform
fft_result = np.fft.rfft(new_df['I-CIP'])
frequencies = np.fft.rfftfreq(len(new_df['I-CIP']), d=1)  # d is the sample spacing

# Plotting the power spectrum
power_spectrum = np.abs(fft_result)
plt.plot(frequencies, power_spectrum)
plt.title('Fourier Analysis - Power Spectrum')
plt.xlabel('Frequency')
plt.ylabel('Amplitude')
plt.show()

From the power spectrum in the fourier analisis above, is observed a large spike near frequency = 0 indicates that most of the variance in the I-CIP data is at very low frequencies. This corresponds to long-term trends or very slow changes in the data.
This could suggest that there is a strong trend or very low-frequency component in the data, which could represent overall market behavior rather than short-term cycles.

Beyond the initial spike, the amplitudes drop significantly and remain low as the  frequencies increase. This suggests that there aren't any strong, regular high-frequency cycles (such as daily or weekly cycles) in the data, or that these patterns are subtle compared to the low-frequency trend.

### Detrending I-CIP 

The plots above show us the trend components to I-cIp prices, and the weekly, quaterly fluctuations around the same periods.
Below, as an attemot to isolating short term fluctuations, since this dataset is relatively short in terms of historical data,by isolating the trend component, is possible to highlight the daily seasonal variations. 

This detrended series allows for a clearer analysis of short-term volatility and cyclical patterns in the I-CIP prices, without the influence of overall market trends, making it useful for exploring potential seasonal behaviors and other short-term irregularities.

In [ ]:
# Using statmodels: Subtracting the Trend Component.

# Ensure the index is set correctly as a datetime index
#new_df.index = pd.to_datetime(new_df.index)

# Now decompose with the known period
result_mul = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=5, extrapolate_trend='freq')

# Subtract the trend from the original data to detrend
detrended = new_df['I-CIP'] - result_mul.trend

# Now you can plot the detrended data
plt.figure(figsize=(12, 6))
plt.plot(detrended, label='Detrended Data')
plt.title('I-CIP prices detrended by subtracting the trend component')
plt.legend()
plt.show()

### Deseasonalized prices

In [ ]:

# Perform seasonal decomposition
result_mul = seasonal_decompose(new_df['I-CIP'], model='multiplicative', period=5, extrapolate_trend='freq')

# Deseasonalize
deseasonalized = new_df['I-CIP'] / result_mul.seasonal

# Plot the deseasonalized data
plt.figure(figsize=(12, 6))
plt.plot(deseasonalized, label='Deseasonalized Data')
plt.title('I-CIP values Deseasonalized', fontsize=16)
plt.legend()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

plot_acf(detrended.dropna(), lags=30)
plt.title("Autocorrelation of Detrended Data")
plt.show()

In [ ]:
# ADF Test
adf_result = adfuller(detrended.dropna())
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])

# KPSS Test
kpss_result = kpss(detrended.dropna(), regression='c')
print('KPSS Statistic:', kpss_result[0])
print('p-value:', kpss_result[1])

## 2.9 Comparing standartisation vs Normalisation vs Differentiation

This section will focus on experimwenting different data preprocessing to:
- obtain a detrended version of the dataset to find more optimal acf/pacf correlation between the values. 
- using data transformation can help to better visualise the seasonality when compared to the same lags. 


 the decomposition suggests that yes, there is  a repeated pattern, which could be further investigated by analyzing the ACF with a higher number of lags. This could help to confirm whether the seasonal pattern seen in the decomposition plot also appears as periodic spikes in the ACF plot, which is what would be typically expect for seasonal data.

In [ ]:
# copy the dataframe of only icip prices  to a new variable
df = new_df['I-CIP']
df = pd.DataFrame(df)

print(df.info())

df.head()


In [ ]:
# Standardization using the mean value
mean_price = df['I-CIP'].mean()
std_price = df['I-CIP'].std()
df['price_standardized'] = (df['I-CIP'] - mean_price) / std_price



In [ ]:
# Normalization using min/max scale
min_price = df['I-CIP'].min()
max_price = df['I-CIP'].max()
df['price_normalized'] = (df['I-CIP'] - min_price) / (max_price - min_price)


#Min-max normalisation has one notable drawback: itstruggles to deal with outliers

#Source: Binita Kumari and Tripti Swarnkar 
    ##International Journal of Advanced Technology and Engineering Exploration, Vol 8(83)1339

In [ ]:
# Differencing
df['price_diff'] = df['I-CIP'].diff()

# Dropping missing values (first rows become nan by defautl)
df.dropna(inplace=True)


In [ ]:
df.head()

## a. Visualising the Standardization and Normalization aplications

In [ ]:
# Plot histogram for original data
fig_hist_original = px.histogram(df, x='I-CIP', title='Histogram: Original Data')
fig_hist_original.show()

# Plot histogram for standardized data
fig_hist_standardized = px.histogram(df, x='price_standardized', title='Histogram: Standardized Data')
fig_hist_standardized.show()

# Plot histogram for normalized data
fig_hist_normalized = px.histogram(df, x='price_normalized', title='Histogram: Normalized Data')
fig_hist_normalized.show()

# Plot histogram for diff data
fig_hist_normalized = px.histogram(df, x='price_diff', title='Histogram: Diff Data')
fig_hist_normalized.show()

# Plot line plot for original data vs differenced data
fig_line = go.Figure()
fig_line.add_trace(go.Scatter(x=df.index, y=df['I-CIP'], mode='lines', name='Original Data'))
fig_line.add_trace(go.Scatter(x=df.index, y=df['price_diff'], mode='lines', name='Differenced Data'))
fig_line.update_layout(title='Line Plot: Original Data vs Differenced Data', xaxis_title='Date', yaxis_title='Price')
fig_line.show()


The first histograms with standardised and normalised values only set the new scales for the prices, but the distribution and overall statistical nature of the data remains the same as the original. However in the diff prices, is clear to see how they change the distribution to a normal dist and reduces the standard deviation from 11 to 2. Below we can also compare the values to see how they behave before and after the first diff. However, to be sure that one diff should be sufficient before moving to modeling, we can check the adf and kpss once again to see the stationarity level. 

-  Differencing removes the trend component from the data, making it stationary. Real price values may exhibit trends over time, which can make modeling more challenging. These values represent the the changes or fluctuations in the variable from one time point to the next.


<i>" Differenced values are often used in conjunction with autoregressive integrated moving average (ARIMA) models, which are well-suited for stationary time series. Forecasting: Differenced values can lead to forecasts of the changes in the variable rather than the actual values themselves. Forecasts based on real price values predict future levels of the variable directly."<i/> https://medium.com/towards-data-science/forecasting-time-series-data-stock-price-analysis-324bcc520af5

### b. ADF/KPPS tests for diff values - checking stationarity again

ADF eith pvalue quite small (4.99e-11) nearly zero which is below the statistical significane and therefore we reject the null hypotesis for non-stationarity. 

KPPS p value 0.1 (>0.05)
Both the ADF and KPSS tests support that the differentiated series is stationary. whihc is also complemented by the barplots above showing the new distribution of diff values.

In [ ]:
# ADF Test
result = adfuller(df['price_diff'], autolag='AIC')
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
for key, value in result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

# KPSS Test
result = kpss(df['price_diff'], regression='c')
print('\nKPSS Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in result[3].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

## 3. Data Preparation for Regression models

# Regression Line>> Finding the trend of the original prices

The trend slope in regression tasks is obtainned by taking the **difference between two data points and dividing it by the amount of time between them.**


a positive trend indicates the future prices tend to increase

In [ ]:
from sklearn.linear_model import LinearRegression

# Extract features (dates) and target variable (prices)
X = new_df.index.to_julian_date().values.reshape(-1, 1)
y = new_df['I-CIP'].values

# Fit linear regression model
lr_model = LinearRegression()
lr_model.fit(X, y)

# Get the slope (trend) of the regression line
trend = lr_model.coef_[0]
print("Trend (slope) of the regression line:", trend)

#### Visualising the trend line

The positive trend of 0.1413 in your regression line indicates that, on average, the I-CIP price increases by 0.1413 units per time step, over time, there is a steady upward movement in prices, reflecting a long-term increasing trend in the I-CIP values.

This also can inidcate models like SARIMA and LSTM could be more apropriate choices as they include a trend componetnet in their structure, as the data exhibits a clear upward movement.

In [ ]:

# Plot the original data
plt.figure(figsize=(10, 6))
plt.plot(new_df.index, new_df['I-CIP'], label='Original Data')

# Plot the regression line
plt.plot(new_df.index, lr_model.predict(X), color='red', linestyle='--', label='Regression Line')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('I-CIP Price')
plt.title('Trend Analysis with Linear Regression')
plt.legend()
plt.grid(True)

# Show plot
plt.show()

### How about using my detrended dataframe?


In [ ]:
# Extract features (dates) and target variable (prices)
X = detrended.index.to_julian_date().values.reshape(-1, 1)
y = detrended.values

# Fit linear regression model
lr_model = LinearRegression()
lr_model.fit(X, y)

# Get the slope (trend) of the regression line
trend = lr_model.coef_[0]
print("Trend (slope) of the regression line:", trend)

In [ ]:

# Plot the original data
plt.figure(figsize=(10, 6))
plt.plot(detrended.index, detrended, label='Detrended Data')

# Plot the regression line
plt.plot(detrended.index, lr_model.predict(X), color='red', linestyle='--', label='Regression Line')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('I-CIP Price')
plt.title('Trend Analysis with Linear Regression')
plt.legend()
plt.grid(True)

# Show plot
plt.show()

In [ ]:
detrended.info()

## 3.1 Split train/test
Checking for optimal ratio between training and testing 

This  time series data has observations recorded only on weekdays (Monday to Friday), so it's important to account for this when splitting the data for training and testing following a sequential splitting (also the data needs to be sorted chronologically by the timestamps). To ensure a proper split for time series data, data scientists typically create a training set to contain observations from the earlier dates and the testing set to contain observations from the later dates. 


- for this first attemp the origibal  values will be used for modeling.and a second attempt with diff values to see if there is any different outcomes. 

While real price values capture the actual levels of the variable of interest, differenced values are often preferred for time series analysis when stationarity is desired and when modeling changes or fluctuations in the data. (Thakar, 2020)

On further sections data imputation techniques are planned to be used to fill all dates and have a more complete dataset. using moving average and foward fill techniques. 


In [ ]:
# checking the dates range before splitting
dates_array = df.index

# Print the array of dates
print("Array of Dates (Index):")
print(dates_array)



**double check fo any missing dates in the index**

In [ ]:
# Generate a date range for 366 days from the start of your data
# Adjust the period accordingly if you have data spanning multiple years or a different time frame
start_date = df.index.min()
end_date = df.index.max()  


# Generate a range of business days within this period
business_days = pd.bdate_range(start=start_date, end=end_date)

# Now compare the business_days with your DataFrame's index to find out missing dates
missing_dates = business_days.difference(df.index)

print(f"Total number of expected business days: {len(business_days)}")
print(f"Total number of actual days in data: {df.shape[0]}")
print(f"Total number of missing dates: {len(missing_dates)}")
print("Missing dates are:")
print(missing_dates)

## Splitting into train/test sets 

### Following the 70/15/15 ratio

70% for training
15% for validation
15% for testing

considering a total of 433 observations


In [ ]:
# Define the sizes
train_size = int(len(df) * 0.7)       # 70% for training
val_size = int(len(df) * 0.15)        # 15% for validation
test_size = len(df) - train_size - val_size  # Remaining 15% for testing

# Split the data
train = df.iloc[:train_size]                      # First 303 observations = 70%
validation = df.iloc[train_size:train_size + val_size]  # Next 64 observations = 15%
test = df.iloc[train_size + val_size:]            # Last 66 observations = 15% aprx

# Print the sizes to confirm
print("Training Set Size:", len(train))         
print("Validation Set Size:", len(validation))  
print("Testing Set Size:", len(test))           


Convert the index to a numeric format for use in the regression model.>>> regression models can’t work with datetime indices directly, we convert the indices to numeric values (integer representations of the timestamps) for X_train, X_val, and X_test.

In [ ]:
# Convert the index to numerical representation for regression models
X_train = pd.to_numeric(train.index).values.reshape(-1, 1)
y_train = train['I-CIP']

X_val = pd.to_numeric(validation.index).values.reshape(-1, 1)
y_val = validation['I-CIP']

X_test = pd.to_numeric(test.index).values.reshape(-1, 1)
y_test = test['I-CIP']

# Print the sizes of each set to verify
print("Training Set Size:", len(train))
print("Validation Set Size:", len(validation))
print("Testing Set Size:", len(test))

#### Verify Data Dimensions:
Make sure that X_test and y_test have the same number of rows.

In [ ]:
print(f"X_test shape: {X_test.shape}")

print(f"y_test shape: {y_test.shape}")

print(f"y_val shape:{y_val.shape}")

#####  Attempt to create a storage for all metrics from different models 


In [ ]:
# Initialize the main dictionary to store metrics for all models
all_metrics = {}

# Define a function to calculate and store metrics
def store_metrics(model_name, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Create a dictionary for the current model's metrics
    error_metrics = {
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape
    }
    
    # Add the current model's metrics to the main dictionary
    all_metrics[model_name] = error_metrics



# Print all metrics
for model, metrics in all_metrics.items():
    print(f"Metrics for {model}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    print("\n")

# Convert the dictionary to a DataFrame for easier comparison IN FUTURE SECTIONS? 

metrics_df = pd.DataFrame(all_metrics).T  # Transpose for better readability
print(metrics_df)


# 4. Regression Models 


### Machine Learning Baseline Models

For this section 

Characteristcs of the data observed thus far can justify the selection of auto-regressive models such as SARIMA(X) and LSTM. However 3 baseline models can be aplied first to create a benchmark and increase the interpretstion of results,.

**Linear Regression:** The simplest model that assumes a linear relationship between the input variables (x) and the single output variable (y). It finds the line (in two dimensions) that best fits the data points. It's easy to understand and interpret <mark> but can be limited in handling complex data patterns without transformations or additions like polynomial features. <mark/> doesnt consider fluctuations and seasonality
    
    
**Decision Tree Regressor** This model uses a decision tree to go from observations about an item  (branches) to conclusions about the item's target value (leaves). It's a simple and interpretable model (can be susceptible to overfitting).

**Random Forest Regressor:** An combination of  methods that fits a number of decision tree regressors on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. It's more robust and accurate than a single decision tree.


**Gradient Boosting Regressor:** An ensemble technique that builds models sequentially, each new model correcting errors made by previous models. It combines weak predictive models to create a strong predictor. Gradient boosting can be used for both regression and classification tasks and is known for its effectiveness, especially with non-linear data.


**SVR (Support Vector Regression):**  uses the principles of support vector machines for regression tasks. It tries to fit the error within a certain threshold and is robust to outliers. <mark> SVR can be used for both linear and non-linear data.<mark/> 

Sources: 
    
[https://towardsdatascience.com/a-beginners-guide-to-regression-analysis-in-machine-learning-8a828b491bbf
https://otexts.com/fpp2/regression.html]
    

    
### 4.1 Linear Regression

In [ ]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:


# Make predictions on the validation set
val_predictions = model.predict(X_val)

# Calculate and print evaluation metrics for the validation set
val_mse = mean_squared_error(y_val, val_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
val_rmse = np.sqrt(val_mse)
val_mape = np.mean(np.abs((y_val - val_predictions) / y_val)) * 100

print("Validation Set Performance:")
print("Mean Squared Error (MSE):", val_mse)
print("Mean Absolute Error (MAE):", val_mae)
print("Root Mean Squared Error (RMSE):", val_rmse)
print("Mean Absolute Percentage Error (MAPE):", val_mape, "%")

# After evaluating on the validation set, make predictions on the testing set
test_predictions = model.predict(X_test)

# Calculate and print evaluation metrics for the testing set
test_mse = mean_squared_error(y_test, test_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mape = np.mean(np.abs((y_test - test_predictions) / y_test)) * 100

print("\nTesting Set Performance:")
print("Mean Squared Error (MSE):", test_mse)
print("Mean Absolute Error (MAE):", test_mae)
print("Root Mean Squared Error (RMSE):", test_rmse)
print("Mean Absolute Percentage Error (MAPE):", test_mape, "%")

# Plot actual vs. predicted values for both validation and testing sets
plt.figure(figsize=(12, 6))
plt.plot(validation.index, y_val, label='Validation Actual')
plt.plot(validation.index, val_predictions, label='Validation Predicted')
plt.plot(test.index, y_test, label='Testing Actual')
plt.plot(test.index, test_predictions, label='Testing Predicted')
plt.xlabel('Date')
plt.ylabel('I-CIP')
plt.title('Actual vs. Predicted Values for Validation and Testing Sets')
plt.legend()
plt.show()


linear regression= not the most appropriate since  the outcomes will not capture the true trend in the prices.
The high error metrics ((MSE): 207, (RMSE): 45.550 and (MAPE): 20.00) ilustrate how different the predictions are from the original values. the validation/ttesting sets are with a very flat line showing the model is not learning the patterns in data. 





In [ ]:
# Example usage for Linear Regression
model_name = "Linear Regression"
# After fitting the model, get predictions
predictions_lr = lr_model.predict(X_test)
store_metrics(model_name, y_test, predictions_lr)

In [ ]:
df.head()

## 4.2 Random Forrest 


In [ ]:
df.head()

### 1. Feature engineering for Random Forest

Adding new time-based features like **day of the week, week of the year, month, and quarter** to the dataset, that can help the model capture any potential seasonality or cyclical patterns in the data and is a step further from the linear regression. This will hopefully decrease the error measures. Lazzeri (2021) [https://medium.com/data-science-at-microsoft/introduction-to-feature-engineering-for-time-series-forecasting-620aa55fcab0]

In [ ]:
# adding day of the week, week of the year, month, and quarter to df
df['DayOfWeek'] = df.index.dayofweek
df['WeekOfYear'] = df.index.isocalendar().week  # Use isocalendar().week to get the week number
df['Month'] = df.index.month
df['Quarter'] = df.index.quarter

# Define features and target variable
features = ['DayOfWeek', 'WeekOfYear', 'Month', 'Quarter']
X = df[features]
y = df['I-CIP']

In [ ]:
df.head()

### 2. Data split 70/15/15

Splitting data nto training, validation, and testing sets using an 70/15/15 ratio. 

The validation set allows to assess model performance before final testing. 

In [ ]:
# First split off the test set (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42) #42 as default

# Then split the remaining 85% into training (70%) and validation (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)  # 15% of 85% is ~15%

### 3. Train Model _ Random Forest REgressor 

In [ ]:

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

### 4. Evaluate Validation Set

still shows a high error for the model ?

In [ ]:
val_predictions = model.predict(X_val)

# Evaluate the model on the validation set
val_mse = mean_squared_error(y_val, val_predictions)
val_mae = mean_absolute_error(y_val, val_predictions)
val_rmse = np.sqrt(val_mse)
val_mape = np.mean(np.abs((y_val - val_predictions) / y_val)) * 100

print("Validation Set Performance:")
print("Mean Squared Error (MSE):", val_mse)
print("Mean Absolute Error (MAE):", val_mae)
print("Root Mean Squared Error (RMSE):", val_rmse)
print("Mean Absolute Percentage Error (MAPE):", val_mape, "%")

### 5.Evaluation of Testing set 

### Actual vs. Predicted Values

error metrics are already smaller than LR however the predicted values are quite far from the actual values, this means there is room for improvement in this model



In [ ]:
# Make predictions on the testing set
test_predictions = model.predict(X_test)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test, test_predictions)
test_mae = mean_absolute_error(y_test, test_predictions)
test_rmse = np.sqrt(test_mse)
test_mape = np.mean(np.abs((y_test - test_predictions) / y_test)) * 100

print("\nTesting Set Performance:")
print("Mean Squared Error (MSE):", test_mse)
print("Mean Absolute Error (MAE):", test_mae)
print("Root Mean Squared Error (RMSE):", test_rmse)
print("Mean Absolute Percentage Error (MAPE):", test_mape, "%")



plt.figure(figsize=(10, 6))
plt.scatter(y_test, test_predictions, color='blue', alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted Values (Random Forest)')
plt.show()

### 6. Improving the model

possible further steps for improve the RF model:
- using GridSearchCv for a Hyperparameter Tuning for Random Forrest
- trying modeling with data in a different scale (using standardised prices)
- add lag features (??) 
- use a different approach for cross validation with a TIMESERIES SPLIT from sklearn



In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
for train_index, val_index in tscv.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    model.fit(X_train, y_train)
    val_predictions = model.predict(X_val)
    print("Validation MSE:", mean_squared_error(y_val, val_predictions))

### 7.  Random Forest Model Using price_standardized with Time Features


In [ ]:
# Define features and target
features = ['DayOfWeek', 'WeekOfYear', 'Month', 'Quarter']
X = df[features]
y = df['price_standardized']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate model
predictions = rf_model.predict(X_test)
# Calculate error metrics
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100


# Print error metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape, "%")

# Store error metrics in a dictionary for later comparison
error_metrics = {
    'Model': 'Random Forest',
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'MAPE': mape
}


Visualize Actual vs. Predicted Values

In [ ]:
# Plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predictions, color='blue', alpha=0.5, label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Ideal Fit')
plt.xlabel('Actual Price Standardized')
plt.ylabel('Predicted Price Standardized')
plt.title('Actual vs. Predicted Values (Random Forest)')
plt.legend()
plt.show()

### 8. Hyperparameter tuning for Random Forest


Best Parameters:
- 'max_depth': None, 
- 'min_samples_leaf': 4, 
- 'min_samples_split': 10, 
- 'n_estimators': 300}
- Best Score: -0.0034037662152510474


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameters grid for RandomForestRegressor
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    #'max_features': ['auto', 'sqrt', 'log2'] was showing excessive warnings? feature discontinued from sklearn!
}

# Initialize RandomForestRegressor
rf = RandomForestRegressor()

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)  # Assuming X_train and y_train are your training features and target

# Get the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [ ]:
# Best parameters from grid search
best_params = {
    'max_depth': None,
    'max_features': 'auto',
    'min_samples_leaf': 4,
    'min_samples_split': 10,
    'n_estimators': 300
}

# Initialize the RandomForestRegressor with the best parameters
regressor = RandomForestRegressor(**best_params)

# Fit the model to your data
# X_train and y_train should be your training data
regressor.fit(X_train, y_train)

# Now the model is ready to make predictions or be evaluated further
predictions = regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Error:", mae)



In [ ]:
#  plot of actual vs predicted values
plt.figure(figsize=(15, 7))

# Plot actual test data as points only (no lines)
plt.plot(y_test.index, y_test, label='Actual Test Data', color='orange', marker='o', linestyle='', markersize=6)

# Plot predictions as individual points with a different style
plt.plot(y_test.index, predictions, label='Random Forest Predictions', color='blue', marker='x', linestyle='', markersize=6)

# Title and labels
plt.title('Random Forest Predictions vs Actual Test Data')
plt.xlabel('Date')
plt.ylabel('Target Variable')

# Add legend
plt.legend()

# Rotate x-axis labels for readability
plt.xticks(rotation=45)

# Show plot
plt.tight_layout()  # Adjust layout for better spacing
plt.show()

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, train_scores, validation_scores = learning_curve(
    regressor, X_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), cv=5, scoring='neg_mean_squared_error')

train_scores_mean = -train_scores.mean(axis=1)
validation_scores_mean = -validation_scores.mean(axis=1)

plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, validation_scores_mean, 'o-', color="g", label="Cross-validation score")
plt.title("Learning Curve")
plt.xlabel("Training Set Size")
plt.ylabel("MSE")
plt.legend(loc="best")
plt.show()

## 4.3 SARIMA

SARIMA and LSTM perfom on a separate notebook to avoid misplacing the variables and have the notebook better structured.



In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
train

In [ ]:
# Define SARIMA model parameters

p = 1  # Non-seasonal AR order
d = 1  # Non-seasonal differencing
q = 1  # Non-seasonal MA order
P = 1  # Seasonal AR order
D = 1  # Seasonal differencing
Q = 1  # Seasonal MA order
m = 21  # Seasonal period ( for monthly seasonality)

# Initialize the model
sarima_model = SARIMAX(train['I-CIP'],
                       order=(p, d, q),
                       seasonal_order=(P, D, Q, m),
                       enforce_stationarity=False,
                       enforce_invertibility=False)

# Fit the model
sarima_result = sarima_model.fit()

# Summary of the model
print(sarima_result.summary())

# Make predictions on the test set
predictions = sarima_result.get_forecast(steps=len(test))
predicted_means = predictions.predicted_mean

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(train, label='Training Data')
plt.plot(test, label='Actual Test Data')
plt.plot(test.index, predicted_means, label='SARIMA Predictions')
plt.legend()
plt.show()

# Calculate error metrics
mse = mean_squared_error(test, predicted_means)
mae = mean_absolute_error(test, predicted_means)
rmse = np.sqrt(mse)

# Print the error metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')

In [ ]:
# Define your parameters (modify p, d, q, P, D, Q as needed)
p, d, q = 1, 1, 1
P, D, Q, m = 1, 1, 1, 12  # Assuming monthly seasonality

# Select the target column 'I-CIP')
target_column = ['price_diff']
train_target = train[target_column]
test_target = test['price_diff']

# Initialize the SARIMAX model
sarima_model = SARIMAX(train_target,
                       order=(p, d, q),
                       seasonal_order=(P, D, Q, m),
                       enforce_stationarity=False,
                       enforce_invertibility=False)

# Fit the model
sarima_result = sarima_model.fit()

# Summary of the model
print(sarima_result.summary())

# Make predictions on the test set
predictions = sarima_result.get_forecast(steps=len(test_target))
predicted_means = predictions.predicted_mean

# Plotting the results
plt.figure(figsize=(10, 5))
plt.plot(train, label='Training Data')
plt.plot(test_target, label='Actual Test Data')
plt.plot(test_target.index, predicted_means, label='SARIMA Predictions')
plt.legend()
plt.show()

# Calculate error metrics
mse = mean_squared_error(test_target, predicted_means)
mae = mean_absolute_error(test_target, predicted_means)
rmse = np.sqrt(mse)

# Print the error metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Root Mean Squared Error (RMSE): {rmse}')


Regression Tree, SVM and Linear
